<a href="https://colab.research.google.com/github/gillesretiere/langdeck_farm/blob/main/langdeck_farm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import pandas as pd
pd.set_option("display.width",1000)
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import re
import string
import unicodedata

# Gspread  API for Google sheets
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall -y gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting

#==== Google Drive 
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/langdeck/
from google.colab import files
from google.colab import auth
auth.authenticate_user()

#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()

#====== GSpread is a Python API for Google Sheets.================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
gc = gspread.authorize(creds)


!export PYTHONIOENCODING=utf8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 358.7 kB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.0.8
Uninstalling gspread-dataframe-3.0.8:
  Successfully uninstalled gspread-dataframe-3.0.8
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
/content/drive/MyDrive/langdeck


In [ ]:
def load_df_from_gsheet (wb, table_name):
  try:
    tbl = wb.worksheet(table_name)
    data = tbl.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
  except:
      print ("Table inexistante !")
  return df
# usage : df = load_df_from_gsheet (table_name):

def save_df_to_gsheet (wb, table_name, df):
  try:
    t_ = wb.worksheet(table_name)
    wb.del_worksheet(t_)
  except:
      print ("Onglet inexistant !")
      return False

  wb.add_worksheet(table_name, 1, 1)
  export_sheet = wb.worksheet(table_name)
  set_with_dataframe(export_sheet, df)
  return True

def get_list_languages():
  df_languages = load_df_from_gsheet ("t_on_languages")
  # on convertit en liste de dict
  vk_languages = json.loads(df_languages.to_json(orient="records"))
  for dic in vk_languages:
    # cast de chaine de liste en liste
    dic['vendor-code']=list(eval((dic['vendor-code'])))
  return vk_languages  

def get_df_as_json(table_name):
  df = load_df_from_gsheet (table_name)
  vk = json.loads(df.to_json(orient="records"))
  return vk  

In [ ]:
db_langdeck_url = "https://docs.google.com/spreadsheets/d/1t0PS0r3WrY2WpwS-llXHujRGjTS3H_c5FQvdeyPuJ-U/"
#db-langdeck_farm
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)

db_langdeck_farm_url = "https://docs.google.com/spreadsheets/d/1JMVoxu4duz9d_0yhf0FOi9LjaMw6EE0GZO4Y_VvRAEg/"
#db-langdeck_farm
sheet = db_langdeck_farm_url
wb_farm  = gc.open_by_url(sheet)

## Languages by **Country**

In [ ]:
def col_to_vkdict(cy, vk):
  my_dictionary = {}
  my_list = []
  for t in vk:
    if not t is None:
      my_dictionary = {}
      my_dictionary["language_uid"] = t[6]
      my_dictionary["language_name_fr"] = t[1]
      my_dictionary["popularity"] = t[2]
      if len(t[2])>0:
        my_dictionary["popularity_as_float"] = float(t[2].strip('%'))/100
      else:
        my_dictionary["popularity_as_float"] = 0.0
      my_dictionary["speakers"] = t[3]
      my_dictionary["is_official"] = t[4]
      my_dictionary["language_name_native"] = t[5]
      my_dictionary["language_wals_code"] = t[0]
      my_dictionary["language_name_en"] = t[7]
      my_list.append(my_dictionary)
  return my_list
  


In [ ]:
s="100.00%"

In [ ]:
float(s.strip('%'))/100

1.0

In [ ]:
df = load_df_from_gsheet(wb_farm, "t_rel_ctry_lang") # load source sheet
# groupby key and cast to a dict : each ctry is key and other cols a list of tuples
dt = df.groupby('country_iso2')[['wals_code','language','rank','speakers','official','name_native','language_code_alpha3','name_en']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
# transform dict to df with key as index of df
df = pd.DataFrame.from_dict(dt, orient='index')
# add column 
df["L"]=df.values.tolist()
df = df.reset_index()
df.rename(columns = {'index':'country_iso2'}, inplace=True)
df = df[["country_iso2","L"]]
df["country_languages"] = df.apply(lambda x:col_to_vkdict(x[0],x[1]), axis=1)
df.drop("L", axis=1, inplace=True)

In [ ]:
df

,country_iso2,country_languages
0,AD,"[{'language_uid': 'fre', 'language_name_fr': '..."
1,AE,"[{'language_uid': 'ams', 'language_name_fr': '..."
2,AF,"[{'language_uid': 'bal', 'language_name_fr': '..."
3,AL,"[{'language_uid': 'alb', 'language_name_fr': '..."
4,AM,"[{'language_uid': 'arm', 'language_name_fr': '..."
...,...,...
145,VN,"[{'language_uid': 'vie', 'language_name_fr': '..."
146,XK,"[{'language_uid': 'alb', 'language_name_fr': '..."
147,YE,"[{'language_uid': 'ams', 'language_name_fr': '..."
148,ZA,"[{'language_uid': 'afr', 'language_name_fr': '..."


In [ ]:
# sauvegarde dans nouveau format
df_farm_ctry = load_df_from_gsheet(wb_farm, "countries")
df_farm_ctry.drop("country_languages", axis=1, inplace=True)
df_countries = pd.merge(df_farm_ctry, df, on="country_iso2", how="left")
save_df_to_gsheet(wb_farm, "countries",df_countries)

True

## Countries by **Language**

In [ ]:
def col_to_vkdict(cy, vk):
  my_dictionary = {}
  my_list = []
  for t in vk:
    if not t is None:
      my_dictionary = {}
      my_dictionary["country_iso3"] = t[6]
      my_dictionary["country_iso2"] = t[0]
      my_dictionary["country_name_fr"] = t[1]
      my_dictionary["popularity"] = t[2]
      if len(t[2])>0:
        my_dictionary["popularity_as_float"] = float(t[2].strip('%'))/100
      else:
        my_dictionary["popularity_as_float"] = 0.0      
      my_dictionary["speakers"] = t[3]
      my_dictionary["is_official"] = t[4]
      my_dictionary["official"] = t[5]
      my_dictionary["national_flag"] = t[7]
      my_dictionary["xy_coordinates"] = t[8]      
      my_list.append(my_dictionary)
  return my_list

In [ ]:
df = load_df_from_gsheet(wb_farm, "t_rel_ctry_lang") # load source sheet
# groupby key and cast to a dict : each ctry is key and other cols a list of tuples
dt = df.groupby('language_code_alpha3')[['country_iso2','country','rank','speakers','official','langue officielle','country_iso3','country_flag','country_xy']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()

In [ ]:
# transform dict to df with key as index of df
df = pd.DataFrame.from_dict(dt, orient='index')
# add column 
df["L"]=df.values.tolist()
df = df.reset_index()

In [ ]:
df.rename(columns = {'index':'language_uid'}, inplace=True)
df = df[["language_uid","L"]]

In [ ]:
df.head()

,language_uid,L
0,abb,"[(TD, Tchad, , , FALSE, , tcd, https://res.clo..."
1,abe,"[(LB, Liban, , , FALSE, , lbn, https://res.clo..."
2,abh,"[(BH, Bahreïn, 30.00%, , FALSE, L'arabe bahreï..."
3,abk,"[(GE, Géorgie, 2.53%, 101,000, FALSE, Langue o..."
4,aeg,"[(EG, Égypte, 100.00%, , FALSE, , egy, https:/..."


In [ ]:
df["language_countries"] = df.apply(lambda x:col_to_vkdict(x[0],x[1]), axis=1)

In [ ]:
df.drop("L", axis=1, inplace=True)

In [ ]:
df

,language_uid,language_countries
0,abb,"[{'country_iso3': 'tcd', 'country_iso2': 'TD',..."
1,abe,"[{'country_iso3': 'lbn', 'country_iso2': 'LB',..."
2,abh,"[{'country_iso3': 'bhr', 'country_iso2': 'BH',..."
3,abk,"[{'country_iso3': 'geo', 'country_iso2': 'GE',..."
4,aeg,"[{'country_iso3': 'egy', 'country_iso2': 'EG',..."
...,...,...
132,wlf,"[{'country_iso3': 'gmb', 'country_iso2': 'GM',..."
133,xho,"[{'country_iso3': 'zaf', 'country_iso2': 'ZA',..."
134,zen,"[{'country_iso3': 'mrt', 'country_iso2': 'MR',..."
135,zho,"[{'country_iso3': 'chn', 'country_iso2': 'CN',..."


In [ ]:
# sauvegarde dans nouveau format
df_farm_languages = load_df_from_gsheet(wb_farm, "languages")
df_farm_languages.drop("language_countries", axis=1, inplace=True)
df_languages = pd.merge(df_farm_languages, df, on="language_uid", how="left")


In [ ]:
df_languages

,language_uid,language_iso2,language_iso2_google,language_wals_code,language_name_fr,language_name_native,language_name_en,language_is_available,language_alpha3,language_genus_fr,language_desc,language_countries
0,ams,ar,ar,ams,Arabe,عرب,Arabic,TRUE,ara,Sémitique,L’arabe standard moderne est le nom que la maj...,"[{'country_iso3': 'bhr', 'country_iso2': 'BH',..."
1,eng,en,en,eng,Anglais,English,English,TRUE,eng,Germanique,,"[{'country_iso3': 'bel', 'country_iso2': 'BE',..."
2,tur,tr,tr,tur,Turc,Türk,Turkish,TRUE,tur,Turc,"Le turc appartient à la famille altaïque, grou...","[{'country_iso3': 'bgr', 'country_iso2': 'BG',..."
3,rus,ru,ru,rus,Russe,Русский,Russian,TRUE,rus,Slave,Le russe est la langue maternelle de près de 1...,"[{'country_iso3': 'arm', 'country_iso2': 'AM',..."
4,ukr,uk,uk,ukr,Ukrainien,Українська,Ukrainian,TRUE,ukr,Slave,"L'ukrainien (en ukrainien : українська мова, o...","[{'country_iso3': 'ukr', 'country_iso2': 'UA',..."
5,rom,ro,ro,rom,Roumain,Română,Romanian,TRUE,rom,Langues latines,Seule langue latine à avoir encore aujourd’hui...,"[{'country_iso3': 'rou', 'country_iso2': 'RO',..."
6,hun,hu,hu,hun,Hongrois,Magyar,Hungarian,TRUE,hun,Ougrien,Le hongrois ou la langue hongroise (hongrois :...,"[{'country_iso3': 'hun', 'country_iso2': 'HU',..."
7,tig,ti,ti,tig,Tigrinya,Tigrinya,Tigrinya,TRUE,tig,Sémitique,,"[{'country_iso3': 'eri', 'country_iso2': 'ER',..."
8,alb,sq,sq,alb,Albanais,Shqiptare,Albanian,TRUE,alb,Albanais,L’albanais est une langue d’origine indo-europ...,"[{'country_iso3': 'alb', 'country_iso2': 'AL',..."
9,geo,ka,ka,geo,Géorgien,Ქართული,Georgian,TRUE,geo,Kartvélien,Total des orateurs\n4 103 000\n\nUtilisation p...,"[{'country_iso3': 'geo', 'country_iso2': 'GE',..."


In [ ]:
save_df_to_gsheet(wb_farm, "languages",df_languages)

True

# END

In [ ]:
def col_to_vkdict(cy, vk):
  my_dictionary = {}
  my_list = []
  for t in vk:
    if not t is None:
      my_dictionary = {}
      my_dictionary["language_uid"] = t[0]
      my_dictionary["language_name_fr"] = t[1]
      my_dictionary["popularity"] = t[2]
      my_dictionary["speakers "] = t[3]
      my_dictionary["is_official "] = t[4]
      my_dictionary["language_name_native "] = t[5]
      my_list.append(my_dictionary)
  return my_list
  
df = load_df_from_gsheet(wb, "t_rel_ctry_lang") # load source sheet
# groupby key and cast to a dict : each ctry is key and other cols a list of tuples
dt = df.groupby('country_iso2')[['wals_code','language','rank','speakers','official','name_native']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
# transform dict to df with key as index of df
df = pd.DataFrame.from_dict(dt, orient='index')
# add column 
df["L"]=df.values.tolist()
df = df.reset_index()
df.rename(columns = {'index':'country_iso2'}, inplace=True)
df = df[["country_iso2","L"]]
df["country_languages"] = df.apply(lambda x:col_to_vkdict(x[0],x[1]), axis=1)
df.drop("L", axis=1, inplace=True)
# sauvegarde dans nouveau format
df_farm_ctry = load_df_from_gsheet(wb_farm, "countries")
df_countries = pd.merge(df_farm_ctry, df, on="country_iso2", how="left")
save_df_to_gsheet(wb_farm, "countries",df_countries)

True

In [ ]:
dt = df_rel.groupby('country_iso2')[['wals_code','language','rank','speakers','official','name_native']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()

In [ ]:
dt

NameError: ignored

In [ ]:
df = pd.DataFrame.from_dict(dt, orient='index')

In [ ]:
df["L"]=df.values.tolist()

In [ ]:
df = df.reset_index()

In [ ]:
df.rename(columns = {'index':'country_iso2'}, inplace=True)

In [ ]:
df = df[["country_iso2","L"]]

In [ ]:
df["D"] = df.apply(lambda x:col_to_vkdict(x[0],x[1]), axis=1)

In [ ]:
df.rename(columns = {'D':'country_languages '}, inplace=True)

In [ ]:
df.drop("L", axis=1, inplace=True)

In [ ]:
df

,country_iso2,country_languages
0,AD,"[{'language_uid': 'spa', 'language_name_fr': '..."
1,AE,"[{'language_uid': 'ams', 'language_name_fr': '..."
2,AF,"[{'language_uid': 'tkm', 'language_name_fr': '..."
3,AL,"[{'language_uid': 'grk', 'language_name_fr': '..."
4,AM,"[{'language_uid': 'aze', 'language_name_fr': '..."
...,...,...
145,VN,"[{'language_uid': 'vie', 'language_name_fr': '..."
146,XK,"[{'language_uid': 'alb', 'language_name_fr': '..."
147,YE,"[{'language_uid': 'ars', 'language_name_fr': '..."
148,ZA,"[{'language_uid': 'zul', 'language_name_fr': '..."


In [ ]:
df_farm_ctry = load_df_from_gsheet(wb_farm, "countries")

In [ ]:
df_farm_ctry

,country_uid,country_iso2,country_iso3,country_name_fr,country_name_en,country_name_native,country_national_flag,region_uid,country_region_fr,country_openstreetmap_xy,country_xy,country_summary,country_languages_summary
0,afg,AF,AFG,Afghanistan,Afghanistan,افغانستان,https://res.cloudinary.com/dhc7ovnwk/image/upl...,asi,Asie,https://www.openstreetmap.org/#map=5/33/65&lay...,"33,65",L’Afghanistan est un pays d'Asie centrale sans...,Il existe 40 langues répertoriées dont 2 langu...
1,alb,AL,ALB,Albanie,Albania,Shqipëria,https://res.cloudinary.com/dhc7ovnwk/image/upl...,eur,Europe,https://www.openstreetmap.org/#map=5/41/20&lay...,"41,20","L'Albanie (littéralement ""pays des aigles""), e...","L'albanais est la langue officielle, parlée da..."
2,dza,DZ,DZA,Algérie,Algeria,الجزائر,https://res.cloudinary.com/dhc7ovnwk/image/upl...,afr,Afrique,https://www.openstreetmap.org/#map=5/28/3&laye...,"28,3",L'Algérie est un pays d’Afrique du Nord faisan...,
3,and,AD,AND,Andorre,Andorra,Andorra,https://res.cloudinary.com/dhc7ovnwk/image/upl...,eur,Europe,https://www.openstreetmap.org/#map=5/42.5/1.6&...,"42.5,1.6","L'Andorre, en forme longue la principauté d'An...",
4,ago,AO,AGO,Angola,Angola,Angola,https://res.cloudinary.com/dhc7ovnwk/image/upl...,afr,Afrique,https://www.openstreetmap.org/#map=5/-12.5/18....,"-12.5,18.5",L’Angola est un État du Sud-Ouest de l'Afrique...,La langue officielle est le portuguais.\nDiale...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,yem,YE,YEM,Yémen,Yemen,اليَمَن,https://res.cloudinary.com/dhc7ovnwk/image/upl...,asi,Asie,https://www.openstreetmap.org/#map=5/15/48&lay...,"15,48",,
176,zmb,ZM,ZMB,Zambie,Zambia,Zambia,https://res.cloudinary.com/dhc7ovnwk/image/upl...,afr,Afrique,https://www.openstreetmap.org/#map=5/-15/30&la...,"-15,30",,
177,zwe,ZW,ZWE,Zimbabwe,Zimbabwe,Zimbabwe,https://res.cloudinary.com/dhc7ovnwk/image/upl...,afr,Afrique,https://www.openstreetmap.org/#map=5/-20/30&la...,"-20,30",,
178,fro,FO,FRO,Îles Féroé,Faroe Islands,Føroyar,https://res.cloudinary.com/dhc7ovnwk/image/upl...,eur,Europe,https://www.openstreetmap.org/#map=5/62/-7&lay...,"62,-7",Les Îles Féroé sont un pays constitutif du Roy...,Féroïen et danois


In [ ]:
df_countries = pd.merge(df_farm_ctry, df, on="country_iso2", how="left")

In [ ]:
def get_translation_fr_xx (input, src, dst):
  print (dst)
  from deep_translator import GoogleTranslator
  r = ""
  try:
    r =  GoogleTranslator(source=src, target=dst).translate(input)
  except Exception as e:
    print (f"Erreur {e}")
  return r

In [ ]:
df_t = df.loc[df["expression"].map(lambda x:x.lower())=="bonjour"][["language","expression"]]

In [ ]:
df_t  = pd.merge(df_t, dfl[["trigramme","iso-code"]], left_on="language", right_on="trigramme", how="left")

In [ ]:
df_t

In [ ]:
df_t["trad"]=df_t[["expression","iso-code"]].apply(lambda x:get_translation_fr_xx(x[0],"fr",x[1]), axis=1)

ar
en
tr
ru
uk
ro
hu
sq
ka
hy
fa
ps
fa
es
pl
ti
so
am
de
pt
az
rw
zh-CN
ur
hi
bn
bg
rn
Erreur rn --> No support for the provided language.
Please select on of the supported languages:
{'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', '

In [ ]:
df_t[["trad","trad"]]

,trad,trad
0,صباح الخير,صباح الخير
1,Hello,Hello
2,Merhaba,Merhaba
3,Привет,Привет
4,Привіт,Привіт
5,Buna ziua,Buna ziua
6,Szia,Szia
7,Përshëndetje,Përshëndetje
8,გამარჯობა,გამარჯობა
9,Բարեւ Ձեզ,Բարեւ Ձեզ


In [ ]:
dfl.iloc[0]

language                                                          Arabe
trigramme                                                           ams
uri                        134rEeVux-FvSxPt4EO4OM1M8PnGHa6P1ewtmnsnRfrs
idx                                                                   1
update                                                            FALSE
alphabet                          اآٱأإبتثجحخدذرزسشصضطظعغفقكلمنهةوؤيئىء
credits                                       Sonia ZARROUK, Wafa TAHRI
font-family                                                  DejaVuSans
font-path               /usr/share/fonts/truetype/dejavu/DejaVuSans.ttf
text-direction                                                      rtl
glossary-subtitle                                            عربي فرنسي
font-family-bold                                        DejaVuSans-Bold
unicode_substition                                                   []
meta                  {'title-fr': '', 'title-tr': '', 'subtitle

## Collection  : Languages

In [ ]:
df = load_df_from_gsheet("t_on_languages")
df.rename(columns={"language":"language_name",
                   "trigramme":"language_trigram",
                   "text-direction":"language_text_direction",
                   "wals_code":"language_wals_code",
                   "iso-code":"language_iso_639_1_code",
                   "vendor-code":"language_translator_code",
                   "language_native":"language_name_native"}, inplace=True)
df = df[["language_name","language_trigram","language_text_direction","language_wals_code","language_iso_639_1_code","language_translator_code","language_name_native"]]
df["language_name_en"] = df["language_name"].map(lambda x:get_translation_fr_en(x))
df["language_pivot_languages"] = df.apply(lambda x: [], axis=1)

In [ ]:
df.iloc[0]

language_name                Arabe
language_trigram               ams
language_text_direction        rtl
language_wals_code             ams
language_iso_639_1_code         ar
language_translator_code    ['12']
language_name_native           عرب
language_name_en              Arab
language_pivot_languages        []
Name: 0, dtype: object

In [ ]:
vk = json.loads(df.to_json(orient="records"))

In [ ]:
with open('languages.json', 'w', encoding='utf-8') as f:
    json.dump(vk, f, ensure_ascii=False, indent=4)

Case studies

In [ ]:
df = load_df_from_gsheet("t_ec_meta")

In [ ]:
df = df[["ID","Nom","Domaine","Thème","Language_code","Language","Date","Lieu","Interprète","Statut","Cover","CR","Présentation","Contexte"]]

In [ ]:
df.rename(columns={"ID":"cs_id",
                   "Nom":"cs_name",
                   "Domaine":"cs_domain",
                   "Thème":"cs_theme",
                   "Language_code":"cs_language_trigram",
                   "Language":"cs_language_name",
                   "Date":"cs_date",
                   "Lieu":"cs_location",
                   "Interprète":"cs_interpret",
                   "Statut":"cs_status",
                   "Cover":"cs_cover",                   
                   "CR":"cs_doc",
                   "Présentation":"cs_resume",
                   "Contexte":"cs_context",                        
                   }, inplace=True)

In [ ]:
vk = json.loads(df.to_json(orient="records"))

In [ ]:
with open('case_studies.json', 'w', encoding='utf-8') as f:
    json.dump(vk, f, ensure_ascii=False, indent=4)

In [ ]:
df.iloc[24]

cs_id                                                               EC25
cs_name                                                Etude de cas n°25
cs_domain                                                        Médical
cs_theme                                                Psychiatrie PASS
cs_language_trigram                                                  dar
cs_language_name                                                    Dari
cs_date                                         2022-10-27T09:30:00.000Z
cs_location            Bâtiment Le Tourville\n1er étage\n5 rue Yves-B...
cs_interpret                                           Massouda Hemyatar
cs_status                                                        Réalisé
cs_cover                             Files_Files_/assets/images/EC25.png
cs_doc                 https://docs.google.com/document/d/1wBkYKa2qxy...
cs_resume              La permanence d’accès aux soins de santé (Pass...
cs_context             Un jeune homme réfugié afgha

In [ ]:
vk_L = get_list_languages()

In [ ]:
vk_L

In [ ]:
json_file = 'Divers/med_vac_synthese.json'
try:
  with open(json_file) as vk_dict:
      vk_lang_dict = json.load(vk_dict)
except BaseException as e:
  print(e)
  uri="null"

In [ ]:
vk = get_df_as_json ("t_ec_meta")

In [ ]:
%cd "/content/drive/MyDrive/Corpus ASAMLA/audio/"

/content/drive/MyDrive/Corpus ASAMLA/audio


In [ ]:
for item in vk_lang_dict:
  dossier = item['language'].lower()
  print (dossier)
  if not os.path.exists(dossier):
    os.makedirs(dossier)


arabe
anglais
turc
russe
ukrainien
roumain
hongrois
tigrinya
albanais
géorgien
arménien
dari
pashto
fârsi
azéri
espagnol
amharique
allemand
polonais
somali


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
from google.oauth2 import service_account

In [ ]:
import gspread
gc = gspread.login('gilles.retiere@gmail.com', '49=Angers%13')

AttributeError: ignored

In [ ]:
!pip install epub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for epub: filename=epub-0.5.2-py3-none-any.whl size=16323 sha256=dfcb413e2f99ed806479a45255f7aec04349e284bbcd644253f236d8ccf5a3e3
  Stored in directory: /root/.cache/pip/wheels/73/62/a5/3a589c9e31319e12b2653c225df490decd5d54387fd47cf425
Successfully built epub


In [ ]:
book = epub.open_epub('path/to/my/book.epub')